# Projet:  
- Réalisation d'une ApplicationDasboard avec Streamlit  
à partir de base de données open source sur les transactions 
de cartes de crédits.

- Lien web vers la source des données:  
[credits_cards_transactions](https://www.kaggle.com/datasets/priyamchoksi/credit-card-transactions-dataset)

## Importation des librairies

- Installation des librairies:  
Exécuter dans la cellule:  %pip install -r requirements.txt  
ou  
Exécuter dans le terminal (prompt ou powershell):  pip install -r requirements.txt

- Importation des librairies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import random
import warnings 
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",100)
random.seed(123)

## Chargement des données

In [2]:
# Fonction pour charger les données

def load(file:str, size:int = 10**5, sep:str = ','):
    """--Docstring--
    fonction pour charger les données csv
    tout en optimisant la mémoire du système
    avec les fichiers volumineux. L'option
    size permet de charger les données par
    partition.
    Args:
        file: (string, file.csv or path )
        size: taille de la partition (integer, default = 100000)
              si la mémoire est insuffisante pour importer en un
              bloc.
        sep: séparateur (string, default = <,>)
    """

    try:

        data = pd.read_csv(file, sep=sep, index_col=0)
    
    except MemoryError:

        data = pd.read_csv(file, sep=sep, index_col=0,
                            chunksize = size)
    
    data.index.name = 'index'
    
    return data


In [3]:
# Chargement des données
path = "credit_card_transactions.csv"
financial_data = load(path) 

## Exploration des données

### affichage des données

In [4]:
financial_data.head(5)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,merch_zipcode
index,,,,,,,,,,,,,,,,,,,,,,,
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,28705.0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,NaN
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,83236.0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,NaN
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,22844.0


### description sur les types de variables

In [5]:
financial_data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud', 'merch_zipcode'],
      dtype='object')

In [6]:
# Fonction pour décrire le dataframe

def info(data):

    Information = pd.DataFrame({
        'Variables': data.columns,
        'Type': data.dtypes,
        'Unique_values': data.nunique(),
        'NA_counts': data.isna().sum(),
        'NA_percent%':data.isna().mean().round(4)*100
        }).reset_index(drop=True)

    return Information

# Application sur financial_data
info(financial_data)

,Variables,Type,Unique_values,NA_counts,NA_percent%
0,trans_date_trans_time,object,1274791,0,0.00
1,cc_num,int64,983,0,0.00
2,merchant,object,693,0,0.00
3,category,object,14,0,0.00
4,amt,float64,52928,0,0.00
5,first,object,352,0,0.00
6,last,object,481,0,0.00
7,gender,object,2,0,0.00
8,street,object,983,0,0.00
9,city,object,894,0,0.00


### mise en forme du type des variables

In [7]:
# fonction pour mettre en forme

def conversion(var):

    # conversion des variables de type = <date>
    date_vars = ('trans_date_trans_time', 'dob')

    if var.name in date_vars:
        
        return pd.to_datetime(var, infer_datetime_format = True)


    # conversion des variables type = <objet>
    if (var.dtype=='object') and (var.name not in date_vars):

        # conversion en string
        if var.nunique() > 20:
            return var.astype('string')

        # conversion en catégorie 
        else:
            return var.astype('category')
    
    else:
        return var 


In [8]:
# application de la fonction sur les variables 
financial_data = financial_data.apply(conversion, axis = 0)

### Création de nouvelles variables

- Variables dates

In [9]:
# mettre la variable trans_date_... au format '2020-01-31'
# avec une nouvelle variable

financial_data['date_transaction'] = pd.to_datetime(financial_data['trans_date_trans_time']
                                                                  .dt.strftime('%d-%b-%Y'))

# Extraire les périodes sur les transactions : jour, semaine de l'année, mois, heure

financial_data['year_transaction'] = (financial_data['trans_date_trans_time']
                                                   .dt.year)

financial_data['day_transaction'] = (financial_data['trans_date_trans_time']
                                                   .dt.day.astype(int))

financial_data['week_transaction'] = financial_data['date_transaction'].dt.isocalendar().week

financial_data['month_transaction'] = (financial_data['trans_date_trans_time']
                                                   .dt.month_name().astype('category'))

financial_data['hour_transaction'] = (financial_data['trans_date_trans_time']
                                                   .dt.strftime('%H.%M')).astype(float)

# Créer une variable moment de la journée à partir de heure_transaction

condlist = [financial_data['hour_transaction'].between(6,12.59),
            financial_data['hour_transaction'].between(13,18.59),
            financial_data['hour_transaction'].between(19,23.59),
            financial_data['hour_transaction'].between(0,5.59)
            ]
choicelist = ['Morning: 6h-12h59',
              'Afternoon: 13h-18h59',
              'Evening: 19h-23h59',
              'Night: 00h-5h59'
              ]

financial_data['moment_transaction'] = np.select(condlist, choicelist, default=None)
financial_data['moment_transaction'] = financial_data['moment_transaction'].astype('category')

- Variables Âge, Nom

In [10]:
# Définir la date de référence pour le calcul de l'âge
reference_date = pd.to_datetime('2020-12-31')

# Calcul de l'âge de l'individu à partir de la date de naissance (dob) 
financial_data['age'] = ((reference_date - financial_data['dob']).dt.days/365).astype('int')

# Création de la variable groupe d'âge
financial_data['group_age']=pd.cut(financial_data['age'],
                                   bins=[14,31,46,60,100],
                                   labels=['15-30 ans', '31-45 ans','46-60 ans' ,'+60 ans']
                                  ).astype('category')

# Concaténer first(prénom) et last(nom_de_famille) pour avoir
# nom complet
financial_data['fullname'] = (financial_data['first'] +' '+ financial_data['last']).astype('string')

### Création de la base de données individuelles
- calcul du montant journalier des transactions individuelles 'daily_amount$'
- calcul du nombre de fraudes individuelles constaté sur la carte de crédits 'daily_fraud'
- calcul du montant de perte subi par l'individu sur les fraudes journalières: daily_fraud_cost$ 
- extraction des variables:  
['fullname','age', 'job','city','daily_amount$','daily_fraud', 'daily_fraud_cost$',  
'date_transaction', 'week_transaction', 'month_transaction', 'year_transaction']

In [ ]:
# liste des variables pour les aggrégations
agglist = ['fullname','date_transaction']

# Calcul du montant journalier des transactions
# par personne
financial_data['daily_amount$'] = financial_data.groupby(agglist)['amt'].transform('sum')

# Calcul du nombre de fraude journalier sur les transactions 
# par personne
financial_data['daily_fraud'] = financial_data.groupby(agglist)['is_fraud'].transform('sum')

# Calcul du cout = montant journalier des fraudes sur transactions 
# par personne
financial_data['daily_fraud_cost$'] = (financial_data[financial_data['is_fraud']==1]
                                                            .groupby(agglist)['amt']
                                                            .transform('sum')
                                      )
financial_data['daily_fraud_cost$'].fillna(0, inplace = True)

liste = ['date_transaction', 'week_transaction', 'month_transaction', 'year_transaction',
         'fullname','age', 'job','state','city','daily_amount$','daily_fraud','daily_fraud_cost$']

# Création du dataframe individus
individual_data = (financial_data[liste].drop_duplicates(subset = agglist, ignore_index = True)
                                        .sort_values(by=agglist, ignore_index = True)
                    )
# Exportation des données au format csv
individual_data.to_csv('individual_data.csv', index = False)

# Affichage des données
individual_data[(individual_data['city']=='Huslia') & (individual_data['daily_fraud'] > 0)]

,date_transaction,week_transaction,month_transaction,year_transaction,fullname,age,job,state,city,daily_amount$,daily_fraud,daily_fraud_cost$
378415,2020-05-25,22,May,2020,Thomas Payne,55,"Engineer, civil (consulting)",AK,Huslia,1997.61,4,1997.61
378416,2020-05-26,22,May,2020,Thomas Payne,55,"Engineer, civil (consulting)",AK,Huslia,2582.49,5,2582.49


### Création de la base de données Etat-villes: state_city_data
- calcul du montant journalier des transactions par ville pour chaque état: 'daily_amountUSD'
- calcul du nombre de fraudes journaliers par ville pour chaque état: 'daily_fraud'
- calcul du montant de perte subi par ville sur les fraudes journalières: daily_fraud_cost$
- extraction des variables:  
['state','city','city_pop','daily_amount$','daily_fraud', 'daily_fraud_cost$'
 'date_transaction', 'week_transaction', 'month_transaction', 'year_transaction']

In [ ]:
# liste des variables dates pour les aggrégations
agglist = ['city', 'year_transaction', 'month_transaction', 'date_transaction']

liste = ['date_transaction', 'week_transaction', 'month_transaction', 'year_transaction',
         'state','city','city_pop','daily_amount$','daily_fraud', 'daily_fraud_cost$']

# Calcul du montant journalier des transactions
# par ville pour chaque Etat
financial_data['daily_amount$'] = financial_data.groupby(agglist)['amt'].transform('sum')

# Calcul du nombre de fraude journalier sur les transactions 
# par ville pour chaque Etat
financial_data['daily_fraud'] = financial_data.groupby(agglist)['is_fraud'].transform('sum')

# Calcul du cout = montant journalier des fraudes sur transactions 
# par ville pour chaque Etat
financial_data['daily_fraud_cost$'] = (financial_data[financial_data['is_fraud']==1]
                                                            .groupby(agglist)['amt']
                                                            .transform('sum')
                                      )
financial_data['daily_fraud_cost$'].fillna(0, inplace = True)

# Création du dataframe Etats/Villes
statecity_data = (financial_data[liste].drop_duplicates(subset = agglist, ignore_index = True)
                                        .sort_values(by=['state', 'city', 'date_transaction'],
                                                     ignore_index = True)
                    )
# exporttaion au format csv
statecity_data.to_csv('statecity_data.csv', index = False)

# Affichage des données
statecity_data[statecity_data['daily_fraud'] > 0].head(60)


,date_transaction,week_transaction,month_transaction,year_transaction,state,city,city_pop,daily_amount$,daily_fraud,daily_fraud_cost$
299,2020-05-25,22,May,2020,AK,Huslia,277,1997.61,4,1997.61
300,2020-05-26,22,May,2020,AK,Huslia,277,2582.49,5,2582.49
316,2019-03-22,12,March,2019,AK,Kaktovik,239,4629.91,7,4629.91
317,2019-03-23,12,March,2019,AK,Kaktovik,239,3177.37,5,3177.37
321,2019-01-04,1,January,2019,AK,Wales,145,1343.52,5,1343.52
322,2019-01-05,1,January,2019,AK,Wales,145,4847.77,10,4847.77
1879,2019-04-26,17,April,2019,AL,Birmingham,493806,2949.33,4,2790.89
1880,2019-04-27,17,April,2019,AL,Birmingham,493806,3329.05,7,3153.72
2121,2019-04-07,14,April,2019,AL,Brantley,2566,4204.70,7,4204.70
2122,2019-04-08,15,April,2019,AL,Brantley,2566,3949.96,5,3949.96


## Analyse Statistique des données

- fonction pour tableau Croisé de Statistiques descriptives

In [ ]:
def cross_stat(data:pd.DataFrame, catlist:list, statvars:list):

    """--Docstring--
    fonction pour réaliser des statistiques
    sur des variables croisées.
    
    Args:
         data: le dataframe
         catlist: (list) variables catégorielles
         statvars: (list) variables continues
    
    return a dataframe
    """
    # initialisation d'un tableau vide
    table = pd.DataFrame()
    
    # Création de dictionnaire pour les indexes
    index_mapping = {value: cat for cat in catlist for value in data[cat].unique()}

    
    for var in statvars:

        for cat in catlist:
            
            X = data.groupby(by=cat)[var].agg(min = 'min',
                                              max = 'max',
                                              mean = 'mean',
                                              st_deviation = 'std',
                                              quartile1 = lambda x: x.quantile(0.25),
                                              median = 'median',
                                              quartile3 = lambda x: x.quantile(0.75),
                                              category_size = 'count')

            table = pd.concat([table,X], axis = 0, ignore_index = False)
    
        table.reset_index(names = 'valeurs', inplace = True)
        table['Categories'] = table['valeurs'].map(index_mapping)
        table.set_index(['Categories','valeurs'], inplace = True)
    
        print(f'Tableau Statistiques croisées sur: {var}')
        display(table.round(2))
        print('\n\n')
